The following Jupyter Notebook is to derive the following static catchment attributes (climatographic):

-> p_mean \
-> p_seasonality \
-> snow_frac_daily \
-> high_prec_freq \
-> high_prec_dur\
-> low_prec_freq\
-> low_prec_dur 

In [1]:
import pandas as pd 
import numpy as np 
import os 
import scipy 

In [2]:
def calc_p_mean(df: pd.DataFrame) -> float:
    """Calculates mean precipitation for the whole period"""
    # calculate mean precipitations for each year
    df = df.groupby(df.date.dt.year)["prcp"].mean()
    # return the average
    return df.mean()

In [3]:
def calc_p_seasonality(df: pd.DataFrame) -> float:
    """Calculates precipitation seasonality"""

    df["day_of_year"] = df["date"].dt.strftime("%m-%d")

    average_prcp = df.groupby("day_of_year")["prcp"].mean().reset_index()
    average_prcp.columns = ["day_of_year", "avg_prcp"]
    sorted_days = (
        pd.date_range(start="2019-05-01", end="2020-04-30").strftime("%m-%d").tolist()
    )
    average_prcp["day_of_year"] = pd.Categorical(
        average_prcp["day_of_year"], categories=sorted_days, ordered=True
    )
    average_prcp = average_prcp.sort_values("day_of_year").reset_index(drop=True)

    average_t_mean = df.groupby("day_of_year")["t_mean"].mean().reset_index()
    average_t_mean.columns = ["day_of_year", "avg_t_mean"]
    average_t_mean = average_t_mean.sort_values(by="day_of_year").reset_index(drop=True)
    average_t_mean["day_of_year"] = pd.Categorical(
        average_t_mean["day_of_year"], categories=sorted_days, ordered=True
    )
    average_t_mean = average_t_mean.sort_values("day_of_year").reset_index(drop=True)

    extended_prcp = pd.concat([average_prcp] * 3, ignore_index=True)
    extended_prcp["29_day_avg_prcp"] = (
        extended_prcp["avg_prcp"].rolling(window=29, center=True).mean()
    )
    centered_29_day_avg = extended_prcp.iloc[366:732].reset_index(drop=True)

    extended_t_mean = pd.concat([average_t_mean] * 3, ignore_index=True)
    extended_t_mean["29_day_avg_prcp"] = (
        extended_t_mean["avg_t_mean"].rolling(window=29, center=True).mean()
    )
    centered_29_day_avg_t_mean = extended_t_mean.iloc[366:732].reset_index(drop=True)

    average_prcp["29_day_avg_prcp"] = centered_29_day_avg["29_day_avg_prcp"]
    average_t_mean["29_day_avg_t_mean"] = centered_29_day_avg_t_mean["29_day_avg_prcp"]

    df = pd.merge(average_prcp, average_t_mean, on="day_of_year")

    t_data = df["29_day_avg_t_mean"].values
    p_data = df["29_day_avg_prcp"].values

    N = len(t_data)
    t = np.linspace(0, 2 * np.pi, N)

    t_guess_mean = np.mean(t_data)
    t_guess_amp = (np.max(t_data) - np.min(t_data)) / 2
    p_guess_mean = np.mean(p_data)
    p_guess_amp = (np.max(p_data) - np.min(p_data)) / 2
    guess_phase = 0

    t_optimize_func = lambda x: x[0] * np.sin(t + x[1]) + x[2] - t_data
    p_optimize_func = lambda x: x[0] * np.sin(t + x[1]) + x[2] - p_data

    t_est_amp, t_est_phase, t_est_mean = scipy.optimize.leastsq(
        t_optimize_func, [t_guess_amp, guess_phase, t_guess_mean]
    )[0]
    p_est_amp, p_est_phase, p_est_mean = scipy.optimize.leastsq(
        p_optimize_func, [p_guess_amp, guess_phase, p_guess_mean]
    )[0]

    p_est_amp = p_est_amp / p_est_mean

    p_seasonality = (
        p_est_amp * t_est_amp / abs(t_est_amp) * np.cos(p_est_phase - t_est_phase)
    )

    return p_seasonality

In [15]:
def calc_snow_frac_daily(df: pd.DataFrame) -> float:
    """Calculates the fraction of snow precipitation to the total precipitations for the whole period"""
    # take days where max temperature were less than 0
    df_snow_frac_daily = df[df["t_mean"] < 0]
    # take rows where precipitation is not NaN and more than 0
    df_snow_frac_daily = df_snow_frac_daily[
        (df_snow_frac_daily["prcp"].notna()) & (df_snow_frac_daily["prcp"] > 0)
    ]
    # compute the fraction of snow to the total precipitations amount
    df_snow_frac_daily = df_snow_frac_daily["prcp"].sum() / df["prcp"].sum()
    return df_snow_frac_daily
    # print(df.head())
    # mean_monthly_temp = df["t_mean"].groupby(df.Mnth).mean()
    # mean_monthly_precip = df["prcp"].groupby(df.Mnth).mean()
    # return mean_monthly_precip.loc[mean_monthly_temp < 0].sum() / mean_monthly_precip.sum()

In [5]:
def calc_aridity(df: pd.DataFrame) -> pd.Series:
    """Calculates aridity as ratio of `pet_mean` to `p_mean`"""
    return df["pet_mean"] / df["p_mean"]


def calc_high_prec_freq(df: pd.DataFrame, p_mean: float) -> float:
    """Calculates the number of high precipitation days and averages across years"""
    # take the days where precipitations were 5 times the precipitation daily mean
    high_prec_freq_df = df[df["prcp"] >= 5 * p_mean]
    # obtain the number of such days for each year
    high_prec_freq = high_prec_freq_df.groupby(high_prec_freq_df.date.dt.year).size()
    # return the average across years
    return high_prec_freq.mean()


def calc_high_prec_dur(df: pd.DataFrame, p_mean: float) -> float:
    """Calculates the average duration of high precipitation days across all years"""
    # filter rows by the amount of precipitations
    df = df[df["prcp"] >= 5 * p_mean]
    s = df.groupby("year").date.diff().dt.days.ne(1).cumsum()
    df = df.groupby(["year", s]).size().reset_index(level=1, drop=True)
    # find average consecutive days for each year
    df_high_prec_dur_years = df.groupby("year").mean()
    # return average consecutive days across all years
    return df_high_prec_dur_years.mean()

In [9]:
def calc_low_prec_freq(df: pd.DataFrame) -> float:
    """Calculates the number of dry days and averages across years"""
    # take the days where precipitations were less than 1 mm/day
    low_prec_freq_df = df[df["prcp"] < 1]
    # obtain the number of such days for each year
    low_prec_freq = low_prec_freq_df.groupby(low_prec_freq_df.date.dt.year).size()
    # return the average across years
    return low_prec_freq.mean()


def calc_low_prec_dur(df: pd.DataFrame) -> float:
    """Calculates the average duration of high precipitation days across all years"""
    # filter rows by the amount of precipitations
    df = df[df["prcp"] < 1]
    s = df.groupby("year").date.diff().dt.days.ne(1).cumsum()
    df = df.groupby(["year", s]).size().reset_index(level=1, drop=True)
    # find average consecutive days for each year
    df_low_prec_dur_years = df.groupby("year").mean()
    # return average consecutive days across all years
    return df_low_prec_dur_years.mean()

In [ ]:
def process_basin(df: pd.DataFrame, basin_id: str) -> pd.DataFrame:
    """Calculates static attributes for a given basin"""

    # add new column `year`
    df_year = df.copy()
    df_year["year"] = df["date"].dt.year

    df = df_year

    # start calculations
    p_mean = calc_p_mean(df)
    
    p_seasonality = calc_p_seasonality(df)
    snow_frac_daily = calc_snow_frac_daily(df)
    high_prec_freq = calc_high_prec_freq(df, p_mean=p_mean)
    high_prec_dur = calc_high_prec_dur(df, p_mean=p_mean)
    low_prec_freq = calc_low_prec_freq(df)
    low_prec_dur = calc_low_prec_dur(df)

    # return the dataframe with a single basin
    return pd.DataFrame(
        data={
            "basin_id": basin_id,
            "p_mean": p_mean,
            "p_seasonality": p_seasonality,
            "frac_snow_daily": snow_frac_daily,
            "high_prec_freq": high_prec_freq,
            "high_prec_dur": high_prec_dur,
            "low_prec_freq": low_prec_freq,
            "low_prec_dur": low_prec_dur,
        },
        index=[0],
    )

In [14]:
# example with precipitation from ERA5 Land 
clim_static_df = pd.DataFrame()

# processing sample basin 11001
meteo_path = '11001.csv'

'''In this example we have only one basin to process'''
df = pd.read_csv(meteo_path, parse_dates=['date'])
df.rename(columns={'prcp_era': 'prcp'}, inplace=True)

clim_static_df = process_basin(df, meteo_path.split('.')[0])
print(clim_static_df)



'''IF meteo_path is a directory of multiple basins --> uncomment'''
# result = []
#
# for file in os.listdir(meteo_path):
#     df = pd.read_csv(meteo_path + file, parse_dates=['date'])
#     df.rename(columns={'prcp_era': 'prcp'}, inplace=True)
#     result.append(process_basin(df, file.split('.')[0]))

# clim_static_df = pd.concat(result, axis=0)
# print(clim_static_df)


  basin_id    p_mean  p_seasonality  frac_snow_daily  high_prec_freq  \
0    11001  1.317507       0.361732         0.346277       14.913043   

   high_prec_dur  low_prec_freq  low_prec_dur  
0       1.230903     243.304348      4.478745  


'IF meteo_path is a directory of multiple basins --> uncomment'